In [21]:
# Import libraries
from arcgis.gis import GIS
from IPython.display import display
from getpass import getpass

In [22]:
source_password = getpass()
# target_password = getpass()
source = GIS("https://yondr.maps.arcgis.com/", 'colin.langhorn_Yondr', source_password)
# AGOL DEV
# target = GIS('https://yondrdev.maps.arcgis.com/','YondrDev' , 'df1ZDR5vhG2a')
# GISMO
target = GIS('https://gismo.yondrgroup.com/portal/home','yondr.publisher' , '7ZhHZG7ebuMHh4k')

In [26]:
#Get a specific item using item id:
itemid = '9ed8882d5cfe4d0787a58e7b80dae8e1'
items = source.content.search(itemid) 

print(str(len(items)) + " items will be cloned. See the list below:") 
items

def deep_copy_content(input_list):
    for item in input_list:
        try:
            print("Cloning " + item.title)
            copy_list = []
            copy_list.append(item)
            target.content.clone_items(copy_list, copy_data=True, search_existing_items=True)
            print("Successfully cloned " + item.title)
        except Exception as e:
            print(e)
    print("The function has completed")
deep_copy_content(items)

1 items will be cloned. See the list below:
Cloning FiberUpload_2022-09-27-H16-M56
Successfully cloned FiberUpload_2022-09-27-H16-M56
The function has completed


## Copy Items
Below we define a function that you can call in a loop for each item in the dictionary we composed earlier. If the item is a text based item such as a Web Map or a file based item such as a layer package, it downloads the item's data to a temporary directory and uses that for creating the target item during cloning. You can find the [exhaustive list of different items](http://doc.arcgis.com/en/arcgis-online/reference/supported-items.htm) that you can upload to your portal and their corresponding item types from the [REST API documentation](http://resources.arcgis.com/en/help/arcgis-rest-api/index.html#/Items_and_item_types/02r3000000ms000000/). For brevity, this sample covers only a subset of those items. Note, if the item points to a web layer URL, the target item would also point to the same URL.

In [3]:
TEXT_BASED_ITEM_TYPES = frozenset(['Web Map', 'Feature Service', 'Map Service','Web Scene',
                                   'Image Service', 'Feature Collection', 
                                   'Feature Collection Template',
                                   'Web Mapping Application', 'Mobile Application', 
                                   'Symbol Set', 'Color Set',
                                   'Windows Viewer Configuration'])

FILE_BASED_ITEM_TYPES = frozenset(['File Geodatabase','CSV', 'Image', 'KML', 'Locator Package',
                                  'Map Document', 'Shapefile', 'Microsoft Word', 'PDF',
                                  'Microsoft Powerpoint', 'Microsoft Excel', 'Layer Package',
                                  'Mobile Map Package', 'Geoprocessing Package', 'Scene Package',
                                  'Tile Package', 'Vector Tile Package'])

ITEM_COPY_PROPERTIES = ['title', 'type', 'typeKeywords', 'description', 'tags',
                        'snippet', 'extent', 'spatialReference', 'name',
                        'accessInformation', 'licenseInfo', 'culture', 'url']

In [4]:
import tempfile
def copy_item(target, source_item):
    try:
        with tempfile.TemporaryDirectory() as temp_dir:
            item_properties = {}
            for property_name in ITEM_COPY_PROPERTIES:
                item_properties[property_name] = source_item[property_name]

            data_file = None
            
            if source_item.type in TEXT_BASED_ITEM_TYPES:
                # If its a text-based item, then read the text and add it to the request.
                text = source_item.get_data(False)
                item_properties['text'] = text
            
            elif source_item.type in FILE_BASED_ITEM_TYPES:
                # download data and add to the request as a file
                data_file = source_item.download(temp_dir)

            thumbnail_file = source_item.download_thumbnail(temp_dir)
            metadata_file = source_item.download_metadata(temp_dir)

            #find item's owner
            source_item_owner = source.users.search(source_item.owner)[0]
            
            #find item's folder
            item_folder_titles = [f['title'] for f in source_item_owner.folders 
                                  if f['id'] == source_item.ownerFolder]
            folder_name = None
            if len(item_folder_titles) > 0:
                folder_name = item_folder_titles[0]

            #if folder does not exist for target user, create it
            if folder_name:
                target_user = target.users.search(source_item.owner)[0]
                target_user_folders = [f['title'] for f in target_user.folders
                                       if f['title'] == folder_name]
                if len(target_user_folders) == 0:
                    #create the folder
                    target.content.create_folder(folder_name, source_item.owner)
            
            # Add the item to the target portal, assign owner and folder
            target_item = target.content.add(item_properties, data_file, thumbnail_file, 
                                             metadata_file, source_item.owner, folder_name)
            
            #Set sharing (privacy) information
            share_everyone = source_item.access == 'public'
            share_org = source_item.access in ['org', 'public']
            share_groups = []
            if source_item.access == 'shared':
                share_groups = source_item.groups
            
            target_item.share(share_everyone, share_org, share_groups)
            
            return target_item
        
    except Exception as copy_ex:
        print("\tError copying " + source_item.title)
        print("\t" + str(copy_ex))
        return None

In [19]:
# Get all Items in a group
group_items = []
groups = source.groups.search(query = 'title: "Gismo 1.0"')
for group in groups:
    for group_item in group.content():
        print(group_item)
        group_items.append(group_item)

<Item title:"AGOL-GISMO 1.0 Dev Manager App" type:Web Experience owner:yondrgroup1>
<Item title:"AGOL-Gismo 1.0 GCS Client App" type:Web Experience owner:yondrgroup1>
<Item title:"AGOL-Gismo 1.0 Map" type:Web Map owner:yondrgroup1>
<Item title:"FiberCableNetwork" type:Vector Tile Layer owner:yondrgroup1>
<Item title:"GISMO 1.0 Dev Manager App" type:Web Experience owner:yondrgroup1>
<Item title:"Gismo 1.0 GCS Client App" type:Web Experience owner:yondrgroup1>
<Item title:"Gismo 1.0 Map" type:Web Map owner:yondrgroup1>
<Item title:"Gismo1PrintTask" type:Geoprocessing Toolbox owner:yondrgroup1>
<Item title:"GismoSite" type:Feature Layer Collection owner:yondrgroup1>
<Item title:"GlobalDCInfrastruture" type:Feature Layer Collection owner:yondrgroup1>
<Item title:"HostedGismo1Site" type:Feature Layer Collection owner:yondrgroup1>
<Item title:"SubmarineCable" type:Feature Layer Collection owner:yondrgroup1>
<Item title:"Substations _Global" type:Feature Layer Collection owner:yondrgroup1>
<I

In [20]:
# Copy over each item. While doing so, construct a dictionary mapping of source item's ID with target item's ID
source_target_itemId_map = {}
for item in group_items:
    print("Copying {} \tfor\t {}".format(item.title, item.owner))
    target_item = copy_item(target, item)
    if target_item:
        item.itemid = target_item.itemid
    else:
        item.itemid = None

Copying AGOL-GISMO 1.0 Dev Manager App 	for	 yondrgroup1
	Error copying AGOL-GISMO 1.0 Dev Manager App
	list index out of range
Copying AGOL-Gismo 1.0 GCS Client App 	for	 yondrgroup1
	Error copying AGOL-Gismo 1.0 GCS Client App
	list index out of range
Copying AGOL-Gismo 1.0 Map 	for	 yondrgroup1
	Error copying AGOL-Gismo 1.0 Map
	list index out of range
Copying FiberCableNetwork 	for	 yondrgroup1
	Error copying FiberCableNetwork
	list index out of range
Copying GISMO 1.0 Dev Manager App 	for	 yondrgroup1
	Error copying GISMO 1.0 Dev Manager App
	list index out of range
Copying Gismo 1.0 GCS Client App 	for	 yondrgroup1
	Error copying Gismo 1.0 GCS Client App
	list index out of range
Copying Gismo 1.0 Map 	for	 yondrgroup1
	Error copying Gismo 1.0 Map
	list index out of range
Copying Gismo1PrintTask 	for	 yondrgroup1
	Error copying Gismo1PrintTask
	list index out of range
Copying GismoSite 	for	 yondrgroup1
	Error copying GismoSite
	list index out of range
Copying GlobalDCInfrastrutur